## Basic Model Initialization:

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
checkpoint='bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

input = ['Hi my name is pavan ','my inspiration is virat kohli']

tokens = tokenizer(input, padding = True, truncation = True, return_tensors='pt')
print('token_ids:', tokens)

output =  model(**tokens)
output

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


token_ids: {'input_ids': tensor([[  101,  7632,  2026,  2171,  2003,  6643,  6212,   102,     0],
        [  101,  2026,  7780,  2003,  6819,  8609, 12849, 27766,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]])}


SequenceClassifierOutput(loss=None, logits=tensor([[-0.2325, -1.0362],
        [-0.1998, -1.0125]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## Dataset Loading:

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 wh

In [ ]:
from datasets import load_dataset

data = load_dataset('glue','stsb')
data

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 5749
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1379
    })
})

In [ ]:
data['train'][:3]

{'sentence1': ['A plane is taking off.',
  'A man is playing a large flute.',
  'A man is spreading shreded cheese on a pizza.'],
 'sentence2': ['An air plane is taking off.',
  'A man is playing a flute.',
  'A man is spreading shredded cheese on an uncooked pizza.'],
 'label': [5.0, 3.799999952316284, 3.799999952316284],
 'idx': [0, 1, 2]}

In [ ]:
data['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': Value(dtype='float32', id=None),
 'idx': Value(dtype='int32', id=None)}

## Preprocessing Data:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def token_fun(lines):
  return( tokenizer(lines['sentence1'], lines['sentence2'], padding=True, truncation=True))

tokenized_data = data.map(token_fun)
print(tokenized_data.column_names)

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

{'train': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'validation': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask']}


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def token_fun(lines):
  return( tokenizer(lines['sentence1'], lines['sentence2'], padding=True, truncation=True))

tokenized_data = data.map(token_fun,batched = True)

tokenized_data = tokenized_data.remove_columns(['sentence1', 'sentence2','idx'])
tokenized_data = tokenized_data.rename_column('label','labels')
tokenized_data  = tokenized_data.with_format('torch')
tokenized_data['train']

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5749
})

In [ ]:
small_train_data = tokenized_data['train'].select(range(100))
small_train_data

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 100
})

In [ ]:
tokenized_data_1 = tokenizer(data['train']['sentence1'])
tokenized_data_2 = tokenizer(data['train']['sentence2'])

print(tokenized_data_1['input_ids'][0])
print(tokenizer.convert_ids_to_tokens(tokenized_data_1['input_ids'][0]))
tokenized_data_1[0]

[101, 1037, 4946, 2003, 2635, 2125, 1012, 102]
['[CLS]', 'a', 'plane', 'is', 'taking', 'off', '.', '[SEP]']


Encoding(num_tokens=8, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
print(tokenized_data_2['input_ids'][0])
print(tokenizer.convert_ids_to_tokens(tokenized_data_2['input_ids'][0]))
tokenized_data_2[0]

[101, 2019, 2250, 4946, 2003, 2635, 2125, 1012, 102]
['[CLS]', 'an', 'air', 'plane', 'is', 'taking', 'off', '.', '[SEP]']


Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

## Dynamic Padding:

**1.Lets do it with normal padding**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
checkpoint = 'bert-base-uncased'

input = load_dataset('glue','stsb')

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model =  AutoModelForSequenceClassification.from_pretrained(checkpoint)

def token_fun(lines):
  return(tokenizer(lines['sentence1'],lines['sentence2'], padding = 'max_length', truncation = True, max_length=128))


token_data = input.map(token_fun, batched = True)
tokenized_data = token_data.remove_columns(['sentence1','sentence2','idx'])
tokenized_data = tokenized_data.rename_column('label','labels')
tokenized_data = tokenized_data.with_format('torch')



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader

train_data = DataLoader(tokenized_data['train'],batch_size =16, shuffle=True)

for ite, batch in enumerate(train_data):
  print(batch['input_ids'].shape)
  if ite >6:
    break

torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])


we can see all the batch are of same size

**2.Now lets do it with Dynamic padding:**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
checkpoint = 'bert-base-uncased'

input = load_dataset('glue','stsb')

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model =  AutoModelForSequenceClassification.from_pretrained(checkpoint)

def token_fun(lines):
  return(tokenizer(lines['sentence1'],lines['sentence2'], truncation = True))


token_data = input.map(token_fun, batched = True)
tokenized_data = token_data.remove_columns(['sentence1','sentence2','idx'])
tokenized_data = tokenized_data.rename_column('label','labels')
tokenized_data = tokenized_data.with_format('torch')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collat = DataCollatorWithPadding(tokenizer)

train_data = DataLoader(tokenized_data['train'],batch_size =16, shuffle=True, collate_fn=data_collat)

for ite, batch in enumerate(train_data):
  print(batch['input_ids'].shape)
  if ite >6:
    break

torch.Size([16, 46])
torch.Size([16, 55])
torch.Size([16, 70])
torch.Size([16, 68])
torch.Size([16, 57])
torch.Size([16, 42])
torch.Size([16, 54])
torch.Size([16, 67])


Now, we can see we have different batch with different sizes.

In Dynamic Padding each batch is padded to match the longest sequence in that batch.

Instead of padding all sequences to a fixed max_length, dynamic padding ensures that shorter batches use less padding.
Often used with batching strategies like bucketing, where sequences of similar lengths are grouped together.